Toute fonction peut être vue comme une somme infinie d'impulsion de Dirac: $$x(t) = \int_{-\infty}^{\infty} x(\tau) \delta(t-\tau) d\tau$$

Si on connaît la réponse d'un système LTI (Linear and Time Invariant) à une impulsion de Dirac: $$\delta(t) \rightarrow h(t)$$
on peut calculer par linéarité la réponse au signal $f(t)$ quelconque: $$y(t) = \int_{-\infty}^{\infty} x(\tau) h(t-\tau) d\tau = x(t) * h(t)$$
Cette intégrale est appelée produit de convolution de $f$ et $h$et est noté $*$.

C'est ici qu'il est intéressant de passer dans le domaine de Laplace car les produits de convolution deviennent de simple produits:
$$Y(s) = \mathcal{L}[y(t)] = \mathcal{L} [x(t) * h(t)] = \mathcal{L}[x(t)]\cdot\mathcal{L}[h(t)] = X(s) \cdot H(s)$$

$H(t)$ est la fonction de transfère du système LTI.

![](./convolution_and_laplace.png)


In [ ]:
# Avec sympy pour une application directe des maths
import sympy as sp

s = sp.symbols('s')

H = 2 / (s**2 + 0.4*s + 2)
H

In [ ]:
t = sp.symbols('t')

x = sp.Piecewise(
    (0, t <= 10),
    (5 * t - 50, (t > 10) & (t < 30)),
    (5 * 30 - 50, t >= 30)
)
x

In [ ]:
from matplotlib import pyplot as plt

# better plots in jupyter
%config InlineBackend.figure_formats = ['svg']

sp.plot(x, (t, 0, 40))
plt.show()

In [ ]:
X = sp.laplace_transform(x, t, s, noconds=True)
Y = H * X
y = sp.inverse_laplace_transform(Y, s, t)

sp.plot(y, (t, 0, 40))
plt.show()

#### Rendre une expression `sympy` compatible avec `numpy`

In [ ]:
import numpy as np

y_fun = sp.lambdify(t, y, "numpy")
t = np.arange(0, 50, 0.1)
y = y_fun(t)

plt.plot(t, y)
plt.show()

#### avec le package `control`

On travaille directement avec des array `numpy`

In [ ]:
# Avec control pour un des fonctions plus spécialisées
import control as ct

s = ct.tf('s')

H = 2 / (s**2 + 0.4*s + 2)
print(H)

In [ ]:
t = np.arange(0, 50, 0.1)

x = 5 * t - 50
x[t < 10] = 0
x[t >= 30] = 5 * 30 - 50

plt.plot(t, x)
plt.show()

In [ ]:
_, y = ct.forced_response(H, t, x)

plt.plot(t, y)
plt.show()

In [ ]:
ct.pzmap(H, grid=True, title="Pole-Zero Map")
plt.show()

In [ ]:
ct.bode(H)
plt.show()

In [ ]:
ct.nyquist(H) # diagram of the gain versus the phase (where the frequency does not appear)
plt.show()

In [ ]:
ct.nichols(H)
plt.grid()
plt.show()

#### Problème avec `control`: impossible de représenter un vrai temps-mort

Fonction de transfert avec temps-mort: $$e^{-sT_m} H(s)$$

Pas de problème avec `sympy`:

In [ ]:
s, t = sp.symbols('s, t')

H = 2 / (s**2 + 0.4*s + 2)

Tm = 2
Htm = sp.exp(-s*Tm) * H

x = sp.Piecewise(
    (0, t <= 10),
    (5 * t - 50, (t > 10) & (t < 30)),
    (5 * 30 - 50, t >= 30)
)

X = sp.laplace_transform(x, t, s, noconds=True)
Ytm = Htm * X
ytm = sp.inverse_laplace_transform(Ytm, s, t)

Y = H * X
y = sp.inverse_laplace_transform(Y, s, t)

_t = np.arange(0, 40, 0.1)

plt.plot(_t, sp.lambdify(t, y, "numpy")(_t), label="sans temps-mort")
plt.plot(_t, sp.lambdify(t, ytm, "numpy")(_t), label="avec temps-mort")
plt.legend()
plt.show()

#### Temps-mort avec `control`

Le module `control` n'a pas de fonction `exp` et celle de `numpy` ne fonctionne pas:

In [ ]:
s = ct.tf('s')

H = 2 / (s**2 + 0.4*s + 2)

# Htm = np.exp(-s*Tm) * H         # provoque une erreur

Nous sommes obligés de travailler avec une approximation polynomiale:

In [ ]:
num, den = ct.pade(Tm, 5) # 5th-order Padé approximation
dead_time = ct.tf(num, den)

Htm = dead_time * H
print(Htm)

In [ ]:
t = np.arange(0, 50, 0.1)

x = 5 * t - 50
x[t < 10] = 0
x[t >= 30] = 5 * 30 - 50

plt.plot(t, ct.forced_response(H, t, x)[1])
plt.plot(t, ct.forced_response(Htm, t, x)[1])
plt.show()

Attention aux zeros et pôles:

In [ ]:
ct.pzmap(Htm, grid=True, title="Pole-Zero Map avec Padé")
plt.show()

#### Comment ajuster un modèle ?

In [ ]:
t = np.arange(0, 10, 0.1)
noise = np.random.randn(len(t)) * 0.2
x = (0.5*t + 2) + noise

plt.scatter(t, x)
plt.show()

In [ ]:
# Le modèle à ajuster. c-a-d trouver m et p qui font que le modèle colle le mieux aux donnée
def model(m, p, t):
    return m*t + p

# distance entre le modèle et les données. Il faut la minimiser
def cost(params):
    return np.linalg.norm(model(params[0], params[1], t) - x)

from scipy.optimize import minimize

res = minimize(cost, [0, 0], method='Nelder-Mead')
m, p = res.x

plt.scatter(t, x)
plt.plot(t, m*t + p)
plt.show()
print("m =", m)
print("p =", p)